This is https://www.tensorflow.org/tutorials/quickstart/beginner, initially forked from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb, and then extended - to self learn.

## Import TensorFlow

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import pandas as pd

TensorFlow version: 2.8.0


## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("Shapes: x_train", x_train.shape, "; y_train", y_train.shape, "; x_test", x_test.shape, "; ", y_test.shape, "\n")
print("Description of y_train:", pd.DataFrame(y_train).describe(), "\n")
print("y_train[:1] = ", y_train[:1])

Shapes: x_train (60000, 28, 28) ; y_train (60000,) ; x_test (10000, 28, 28) ;  (10000,) 

Description of y_train:                   0
count  60000.000000
mean       4.453933
std        2.889270
min        0.000000
25%        2.000000
50%        4.000000
75%        7.000000
max        9.000000 

y_train[:1] =  [5]


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
print(predictions)

[[ 1.223042   -0.15541793 -0.01329105 -0.394733   -0.7688958  -0.89843255
  -0.52713424 -0.26359808 -0.3087973  -0.12512541]]


The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.34809652, 0.08770859, 0.1011037 , 0.0690413 , 0.04749108,
        0.04172103, 0.06047945, 0.07871547, 0.0752368 , 0.09040616]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

3.1767502

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2904 - accuracy: 0.9158
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1407 - accuracy: 0.9586
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1045 - accuracy: 0.9685
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0865 - accuracy: 0.9735
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0737 - accuracy: 0.9772


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0827 - accuracy: 0.9750 - 472ms/epoch - 2ms/step


[0.08272068202495575, 0.9750000238418579]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
# TODO range = :1 # :3
input = x_test[:1] # TODO range
probabilities = probability_model(input).numpy()
print(probabilities)

[[1.2802366e-07 6.2025618e-10 8.8860397e-06 8.7612734e-06 2.3822172e-11
  7.6987384e-08 4.8728393e-14 9.9998188e-01 4.9781267e-08 1.5493698e-07]]


In [ ]:
test = y_test[:1] # TODO range
print(test)

[7]
